In [ ]:
import pandas as pd
import numpy as np

df_origin = pd.read_csv('../backup/HN_cardinal_1_8.csv')
df_origin.shape

In [102]:
#2020년 은 테스트셋으로 분리
df_train = df_origin[df_origin['year'] != 2020]
df_test = df_origin[df_origin['year'] == 2020]

In [103]:
df_test.shape

(7359, 4025)

In [104]:
# 4~8기 19세 이상
df_train = df_train[(df_train['cardinal_number'] >= 4) & (df_train['cardinal_number'] <= 8) & (df_train['age'] >= 19)]
df_train.shape

(87986, 4025)

In [105]:
df_test = df_test[(df_test['cardinal_number'] >= 4) & (df_test['cardinal_number'] <= 8) & (df_test['age'] >= 19)]
df_test.shape

(6133, 4025)

In [106]:
num_features = ["age",
"ainc",
"HE_PLS_15",
"HE_PLS_30",
"HE_mPLS",
"HE_sbp",
"HE_dbp",
"HE_ht",
"HE_wt",
"HE_wc",
"HE_glu",
"HE_HbA1c",
"HE_insulin",
"HE_chol",
"HE_HDL_st2",
"HE_TG",
"HE_LDL_drct",
"HE_ast",
"HE_alt",
"HE_HB",
"HE_HCT",
"HE_BUN",
"HE_crea",
"HE_WBC",
"HE_RBC",
"HE_Bplt",
"HE_Uacid",
"HE_Uph",
"HE_Usg",
"HE_Ucrea"]

In [107]:
onehot_features = ["region",
"sex",
"occp",
"marri_1",
"tins",
"npins",
"D_2_1",
"DI1_pr",
"DI1_pt",
"DI2_pr",
"DI2_pt",
"DI3_pr",
"DI3_pt",
"DJ4_pr",
"DJ4_pt",
"DE1_pr",
"DE1_pt",
"BH9_11",
"BH1",
"BH2_61",
"LQ4_00",
"LQ1_sb",
"LQ2_ab",
"AC1_yr",
"MH1_yr",
"EC_occp",
"EC_stt_1",
"EC_stt_2",
"EC_wht_0",
"BO1_1",
"BO2_1",
"BD1",
"BS8_2",
"BS9_2",
"LW_ms",
"LW_pr",
"LW_oc",
"HE_rPLS",
"HE_Unitr",
"HE_Upro",
"HE_Uglu",
"HE_Uket",
"HE_Ubil",
"HE_Ubld",
"HE_Uro",
"BM1_0",
"BM1_1",
"BM1_2",
"BM1_3",
"BM1_4",
"BM1_5",
"BM1_6",
"BM1_7",
"BM1_8",
"L_BR_TO",
"L_LN_TO",
"L_DN_TO",
"LK_LB_US",
"LK_LB_EF",
"live_t",
"educ",
"BO1",
"HE_obe",]

In [108]:
label_features = ["incm",
"ho_incm",
"incm5",
"ho_incm5",
"edu",
"cfam",
"house",
"D_1_1",
"DI3_2",
"BD1_11",
"BD2_1",
"BA2_12",
"BA2_13",
# "BP1",
"BS3_1",
"BE3_31",
"BE5_1",
"OR1",
]

In [109]:
y_related =["BP_PHQ_1",
"BP_PHQ_2",
"BP_PHQ_3",
"BP_PHQ_4",
"BP_PHQ_5",
"BP_PHQ_6",
"BP_PHQ_7",
"BP_PHQ_8",
"BP_PHQ_9",
"mh_PHQ_S",
"BP6_10",
"BP6_31",
"DF2_pr",
"DF2_pt", "BP1"]

In [110]:
df_train = df_train[num_features + onehot_features + label_features + y_related]
df_test = df_test[num_features + onehot_features + label_features + y_related]

In [111]:
df_test.shape

(6133, 125)

In [112]:
def manual_preprocess(df:pd.DataFrame, num_features, onehot_features, label_features, y_related):
    # 1) 9, 99, 999 인 녀석들(모름/무응답) null 처리
    df = df.replace([9, 99, 999, 9999], np.nan, inplace=False)

    # 2) mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기
    cond = df["BP_PHQ_1"].notna() & \
            df["BP_PHQ_2"].notna() & \
            df["BP_PHQ_3"].notna() & \
            df["BP_PHQ_4"].notna() & \
            df["BP_PHQ_5"].notna() & \
            df["BP_PHQ_6"].notna() & \
            df["BP_PHQ_7"].notna() & \
            df["BP_PHQ_8"].notna() & \
            df["BP_PHQ_9"].notna()
    
    a = df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]]
    b = a[a <= 3].sum(axis=1)
    b[b >= 1] #125

    df['mh_PHQ_S'] = b[b >=1]

    # 3) 결측값 처리 1 - row 기준
    threshold = 0.3 # threshold for null proportion
    null_percent = df.isnull().mean(axis = 1) # 각 행에 대한 결측값 비율 계산
    df = df[null_percent < threshold]
    
    # 4) 변수 생성 `depressed` 
    cond1 = df['mh_PHQ_S'] >= 10 # mh_PHQ_S - total score 
    cond2 = df['BP_PHQ_9'].isin([1, 2, 3]) # person who chose 1, 2, 3 in 9th question 
    cond3 = df['BP6_10'] == 1 
    cond4 = df['BP6_31'] == 1
    cond5 = (df['DF2_pr'] == 1) & (df['mh_PHQ_S'].isna())

    df[cond1 | cond2 | cond3 | cond4 | cond5].drop_duplicates()
 
    df.loc[cond1 | cond2 | cond3 | cond4 | cond5, 'depressed'] = 1 # 조건 중 하나 이상을 만족하는 행에 'depressed' 변수에 1 값 
    df.loc[~(cond1 | cond2 | cond3 | cond4 | cond5), 'depressed'] = 0 # 조건 중 하나라도 만족하지 않는 행에 0 값

    # 5) 학습에 방해가 될 수 있는 그룹 제거 (우울증에 관한 정보가 없는 경우)
    df.drop(df[df['DF2_pr'].isna() & df['mh_PHQ_S'].isna()].index, inplace=True)
    df = df[(df['DF2_pr'] != 8 |  ~df['mh_PHQ_S'].isna())]

    # 6) 결측값 처리 2 - columns 기준
    threshold = 0.1 # 10% 기준
    null_percent = df.isnull().mean() # 각 열에 대한 결측값 비율 계산
    selected_cols = null_percent[null_percent < threshold].index
    df = df[selected_cols]

    # 7) y_related 변수를 피쳐에서 제외하기
    remained_y = set(df.columns).intersection(set(y_related))
    y = df['depressed'] # y 생성
    X = df.drop(columns=list(remained_y), inplace=False) # X 생성
    X = X.drop(columns=['depressed'], inplace=False)

    # 8) 위의 결측값 처리에서 빠진 column을 각 피쳐 그룹에서 제외하기
    num_features = set(df.columns).intersection(set(num_features))
    onehot_features = set(df.columns).intersection(set(onehot_features))
    label_features = set(df.columns).intersection(set(label_features))

    # 9) label features중 비해당(8) 값을 각각 알맞은 값으로 대체하기
    label_ans_8 = ["DI3_2", "BD1_11", "BD2_11", "BA2_12", "BA2_13", "BP1", "BS3_1","BE3_31", "BE5_1"] # BD2_11 - removed

    # 'DI3_2' 변수의 값이 8인 경우, 해당 값을 0으로 바꾸기
    X['DI3_2'] = X['DI3_2'].apply(lambda x: 3 if x == 8 else x)
    X['BD1_11'] = X['BD1_11'].apply(lambda x: 1 if x == 8 else x)
    # X['BD2_11'] = X['BD2_11'].apply(lambda x: 0 if x == 8 else x)
    X['BA2_12'] = X['BA2_12'].apply(lambda x: 5 if x == 8 else x)
    X['BA2_13'] = X['BA2_13'].apply(lambda x: 5 if x == 8 else x)
    X['BA2_13'] = X['BA2_13'].apply(lambda x: 5 if x == 8 else x)
    # X['BP1'] = X['BP1'].apply(lambda x: 5 if x == 8 else x)
    X['BS3_1'] = X['BS3_1'].apply(lambda x: 0 if x == 8 else x)
    # X['BE3_31'] = X['BE3_31'].apply(lambda x: 5 if x == 88 else x)
    # X['BE5_1'] = X['BE5_1'].apply(lambda x: 5 if x == 88 else x)

    print("X.shape:", X.shape)
    print("y.shape:", y.shape)

    return X, y, list(num_features), list(onehot_features), list(label_features), list(y_related)

In [113]:
trn_X, trn_y, num_features, onehot_features, label_features, y_related = manual_preprocess(df_train, num_features, onehot_features, label_features, y_related)
tst_X, tst_y, num_features, onehot_features, label_features, y_related = manual_preprocess(df_test, num_features, onehot_features, label_features, y_related)

X.shape: (74517, 88)
y.shape: (74517,)
X.shape: (5226, 98)
y.shape: (5226,)


In [114]:
"depressed" in tst_X.columns

False

In [115]:
trn_X.shape, tst_X.shape

((74517, 88), (5226, 98))

In [116]:
tst_X = tst_X[list(trn_X.columns)]
tst_X.shape

(5226, 88)

In [117]:
### 임시 -transform
def transform(X, y, num_features, onehot_features, label_features, y_related, is_testset = False):
    ### 결측값 처리 3 - fillna: KNN
    from sklearn.impute import KNNImputer
    from sklearn.impute import SimpleImputer

    # 1) num_features 결측값 처리 - KNN
    knn_imp = KNNImputer(n_neighbors=5)
    X[num_features] = knn_imp.fit_transform(X[num_features])

    # 2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)
    freq_imp = SimpleImputer(strategy = "most_frequent")
    X[onehot_features] = freq_imp.fit_transform(X[onehot_features])
    X[label_features] = freq_imp.fit_transform(X[label_features])

    # transform array(X) to dataframe
    df_new = pd.DataFrame(X, columns=X.columns) 
    df_new.isna().sum().sort_values(ascending = False)

    ### 인코딩
    # 1. numeric # 1. numeric - scaling 
    from sklearn.preprocessing import MinMaxScaler 
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_new[num_features] = scaler.fit_transform(df_new[num_features])
    
    # 2. onehot -> onehot 인코딩으로 진행
    df_new = pd.get_dummies(df_new, columns=onehot_features) # sklearn OneHotEncoder 사용도 가능함

    ### 샘플링
    if is_testset == False:
        # under sampling.  OVER 만 하기엔 너무 많이 생성해야해서 under 선수행
        from imblearn.under_sampling import RandomUnderSampler

        X = df_new
        rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42) # 0.1 -> 0.5로 변경
        X_under, y_under = rus.fit_resample(X, y)
        X_under.shape, y_under.shape

        # over sampling
        from imblearn.over_sampling import SMOTE

        smote = SMOTE(random_state=42)
        X_over, y_over = smote.fit_resample(X_under, y_under)

        X_over.shape, y_over.shape
    else:
        X_over = df_new
        y_over = y

    return X_over, y_over


In [118]:
trn_X_trasformed, trn_y_transformed = transform(trn_X, trn_y, list(num_features), list(onehot_features), list(label_features), list(y_related))

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: Fu

In [119]:
trn_X_trasformed.shape, trn_y_transformed.shape

((26852, 219), (26852,))

In [129]:
tst_X_transformed, tst_y_transformed = transform(tst_X, tst_y, list(num_features), list(onehot_features), list(label_features), list(y_related), is_testset=True)

/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/ijong-won/miniconda3/envs/torch-nightly/lib/python3.11/site-packages/sklearn/utils/validation.py:767: Fu

In [130]:
tst_X_transformed.shape, tst_y_transformed.shape 

((5226, 208), (5226,))

In [131]:
# testset 에만 있는 필드 제거 for test&comparing method
aa = set(trn_X_trasformed.columns) - set(tst_X_transformed.columns)
trn_X_trasformed = trn_X_trasformed.drop(columns=aa, inplace=False)
trn_X_trasformed.shape

(26852, 208)

In [132]:
# aa = set(tst_X_transformed.columns) - set(trn_X_trasformed.columns)
# trn_X_trasformed = trn_X_trasformed.drop(columns=aa, inplace=False)
# trn_X_trasformed.shape

In [133]:
trn_X_trasformed.to_csv('../data/trn_X_20231211_final_col_01_transformed.csv', index=False)
trn_y_transformed.to_csv('../data/trn_y_20231211_final_col_01_transformed.csv', index=False)

tst_X_transformed.to_csv('../data/tst_X_20231211_final_col_01_transformed.csv', index=False)
tst_y_transformed.to_csv('../data/tst_y_20231211_final_col_01_transformed.csv', index=False)

In [134]:
trn_X_trasformed.shape

(26852, 208)

In [135]:
tst_X_transformed.shape

(5226, 208)

In [136]:
# depressed == 1 개수만큼, depressed == 0 인 것들중에서 랜덤으로 뽑아서 1:1 비율로 맞추기
tst_X_transformed['depressed'] = tst_y_transformed

correct_count = tst_X_transformed[tst_X_transformed['depressed'] == 1]
correct_count = len(correct_count)

zero_indices = tst_X_transformed[tst_X_transformed['depressed'] == 0].sample(n=correct_count).index
zero_indices

one_indices = tst_X_transformed[tst_X_transformed['depressed'] == 1].index
one_indices

tst_X_transformed = tst_X_transformed.loc[zero_indices.union(one_indices)]
tst_X_transformed

tst_y_transformed = tst_X_transformed.loc[zero_indices.union(one_indices)]['depressed']
tst_y_transformed


tst_X_transformed.drop(columns=['depressed'], inplace=True)

In [137]:
tst_X_transformed.shape

(470, 208)

In [138]:
tst_X_transformed.to_csv('../data/tst_X_20231211_final_col_01_transformed_undersampling.csv', index=False)
tst_y_transformed.to_csv('../data/tst_y_20231211_final_col_01_transformed_undersampling.csv', index=False)

In [139]:
print(num_features)
print(onehot_features)
print(label_features)
print(y_related)

['HE_TG', 'ainc', 'HE_ast', 'HE_HCT', 'HE_sbp', 'age', 'HE_glu', 'HE_WBC', 'HE_Usg', 'HE_HB', 'HE_ht', 'HE_Bplt', 'HE_crea', 'HE_Uph', 'HE_RBC', 'HE_BUN', 'HE_HDL_st2', 'HE_wc', 'HE_dbp', 'HE_alt', 'HE_chol', 'HE_wt']
['DI3_pr', 'DJ4_pr', 'DI1_pt', 'BM1_4', 'BH2_61', 'BM1_6', 'BM1_8', 'HE_Uket', 'BM1_5', 'live_t', 'BM1_7', 'BD1', 'educ', 'HE_Ubld', 'sex', 'BO1_1', 'DI2_pr', 'EC_stt_2', 'HE_Upro', 'EC_occp', 'BH9_11', 'D_2_1', 'tins', 'LQ2_ab', 'HE_Uro', 'BS8_2', 'marri_1', 'DE1_pr', 'BM1_2', 'BH1', 'occp', 'BO2_1', 'npins', 'BS9_2', 'DI2_pt', 'DI1_pr', 'DJ4_pt', 'LQ4_00', 'HE_rPLS', 'DE1_pt', 'HE_Unitr', 'BM1_1', 'LQ1_sb', 'EC_stt_1', 'HE_Uglu', 'BM1_3', 'HE_obe', 'BO1', 'HE_Ubil', 'DI3_pt']
['DI3_2', 'BD2_1', 'BS3_1', 'ho_incm', 'incm5', 'cfam', 'incm', 'D_1_1', 'BA2_12', 'BA2_13', 'BE5_1', 'house', 'edu', 'BE3_31', 'BD1_11', 'ho_incm5']
['BP_PHQ_1', 'BP_PHQ_2', 'BP_PHQ_3', 'BP_PHQ_4', 'BP_PHQ_5', 'BP_PHQ_6', 'BP_PHQ_7', 'BP_PHQ_8', 'BP_PHQ_9', 'mh_PHQ_S', 'BP6_10', 'BP6_31', 'DF2_pr'